In [20]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
from time import sleep
import re

DIRDATA = 'data/豆瓣高分电影.xlsx'
DOUBAN_MOVIES = pd.read_excel(DIRDATA, sheet_name='movie', index_col=0)

# crawling movies

In [21]:
"""def douban_movie(movie_id):
    res = None
    url = f'https://movie.douban.com/subject/{movie_id}'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0'}
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
            json_str = soup.find('script', {'type': 'application/ld+json'}).text
            res = json.loads(json_str)
    except requests.exceptions.RequestException as e:
        print(f"\n### Error during the request: {e}")
    finally:
        if res is None:
            print("\n not properly loaded!")
        return res
    
movies = {}    
for nums, movie_id in enumerate(DOUBAN_MOVIES.index[:]):
    print(nums, end='-')
    res = douban_movie(movie_id)
    if res is not None:
        movies[movie_id] = res
    sleep(3)
file_path = f"douban_json/movies.json"
with open(file_path, 'w', encoding='utf-8') as json_file:
    json.dump(movies, json_file) 
movies = pd.DataFrame.from_dict(movies, orient ='index')
movies.index.name= 'movie_id'
movies['name'] = movies.index.map(DOUBAN_MOVIES['电影名'])
movies['description'] = movies.index.map(DOUBAN_MOVIES['电影简介']).str.strip()
file_path = f"douban/movies.csv"
movies.to_csv(file_path)"""
print()

In [22]:
file_path = f"douban/movies.csv"
movies = pd.read_csv(file_path, index_col=0)

cols_str = ['director', 'author', 'actor', 'genre', 'aggregateRating']
movies[cols_str] = movies[cols_str].map(eval)

movies['name'] = movies.index.map(DOUBAN_MOVIES['电影名'])
movies['country'] = movies.index.map(DOUBAN_MOVIES['制片地区'])
movies['description'] = movies.index.map(DOUBAN_MOVIES['电影简介']).str.strip()

file_path = f"douban/movies.csv"
movies.to_csv(file_path)

In [23]:
relations = pd.DataFrame()
for col in ['director', 'author', 'actor']:
    for movie_id in movies.index:
        df = pd.DataFrame(movies.loc[movie_id, col])
        df['movie_id'] = movie_id
        df['movie_role'] = col
        relations = pd.concat([relations, df])    
relations['person_id'] = relations.url.str.extract('(\d+)').astype(np.int64)  
relations[['person_id', 'movie_id', 'movie_role']].to_csv('douban/relatons.csv', index=False)


celebrities = relations[['person_id', 'name']].drop_duplicates().reset_index(drop=True)

celebrities[['name_cn', 'name_en']] = celebrities.name.str.strip().str.replace(' ', '|', 1).str.split('|', expand=True).fillna('')
celebrities.to_csv('douban/persons.csv', index=False)
person_ids = relations['person_id'].unique()
pd.DataFrame(person_ids).to_csv('douban/person_ids.csv', index=False)

In [24]:
"""def douban_person(person_id):
    person = None
    url = f'https://movie.douban.com/celebrity/{person_id}'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0'}
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
            person = pd.DataFrame(
                data= [li.text.strip().split('\n') for li in soup.find_all('div', {'class': 'info'})[0].find_all('li')],
                columns=['title', 'content']
            )
            person['title'] = person['title'].str.replace(' |:', '', regex=True)
            person['content'] = person['content'].str.strip()
            person = person.set_index('title')['content']
            person = person.to_dict()
            person['image'] = soup.find('div', {'class': "nbg"}).find('img').get('src')
    except requests.exceptions.RequestException as e:
        print(f"\n### Error during the request: {e}")
    finally:
        if person is None:
            print("not properly loaded!")
        return person
    
persons = {}
for num, person_id in enumerate(person_ids[:]):
    print(num, end='-')
    person = douban_person(person_id)
    if person is not None:
        persons[person_id] = person
    sleep(1)
    
persons_str = {str(key): value for key, value in persons.items()}
file_path = f"douban/persons.json"
with open(file_path, 'w', encoding='utf-8') as json_file:
    json.dump(persons_str, json_file)
"""
print()

In [32]:
file_path = f"douban/persons.json"
with open(file_path, 'r', encoding='utf-8') as json_file:
    persons = json.load(json_file)

persons = pd.DataFrame.from_dict(persons, orient='index')
persons.index.name = 'movie_id'
persons.index =persons.index.astype(np.int64)

file_path = f"douban/persons.csv"
persons.to_csv(file_path)